# Advanced Portfolio Optimization & Risk Analytics

This notebook demonstrates a robust workflow for constructing and analyzing investment portfolios using modern quantitative finance techniques. The workflow includes:

1. **Configuration & Data Pipeline**: Parameter setup and data processing using advanced estimation methods.
2. **Interactive Optimization**: Determining optimal portfolio weights to maximize risk-adjusted returns.
3. **Risk & Return Simulation**: Projecting future performance and quantifying risk via Monte Carlo simulation.

All core logic is modularized in the accompanying Python modules, ensuring this notebook remains focused on high-level analysis and results.

## 1. Setup and Configuration

### 1.1. Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
from IPython.display import clear_output
from ipywidgets import FloatSlider, IntSlider, interact

from portfolio_analyzer.analysis.backtester import Backtester
from portfolio_analyzer.analysis.monte_carlo_simulator import (
    MonteCarloSimulator,
    SimulationStatisticsCalculator,
)
from portfolio_analyzer.config.config import AppConfig
from portfolio_analyzer.core.portfolio_optimizer import PortfolioOptimizer
from portfolio_analyzer.data import data_fetcher
from portfolio_analyzer.data import input_preparator as ip
from portfolio_analyzer.interactive.session import PortfolioAnalysisSession
from portfolio_analyzer.logging_config import setup_logging
from portfolio_analyzer.reporting.display import (
    display_backtest_summary_html,
    display_optimization_summary_html,
)
from portfolio_analyzer.reporting.plotting import (
    calculate_correlation_matrix,
    plot_backtest_results,
    plot_correlation_heatmap,
    plot_correlation_network,
    plot_efficient_frontier,
)

setup_logging()

sns.set_theme(style="whitegrid", palette="pastel")
sns.set_context("notebook")

2025-07-12 16:16:27 - root - INFO - Logging configured. Console level: INFO, File level: DEBUG


### 1.2. Define Parameters

All key parameters are managed via the [`AppConfig`](../src/portfolio_analyzer/config.py) class. This centralizes control over tickers, date ranges, and model hyperparameters. User-defined views for the Black-Litterman model can also be specified here.

In [1]:
# --- General Model Configuration ---
config = AppConfig.get_instance()
config.monte_carlo.num_simulations = 1000

## 2. Data Pipeline & Input Preparation

The [`input_preparator`](../src/portfolio_analyzer/data/input_preparator.py) module encapsulates the end-to-end data pipeline: fetching historical prices and market capitalizations, computing log returns, and preparing the mean return vector and covariance matrix for optimization.

**Note:** If enabled, the pipeline automatically performs DCF analysis for each asset to generate forward-looking views for the Black-Litterman model. Assets lacking market capitalization default to their implied equilibrium return, blending market-neutral priors with DCF-based views.

Robust estimation techniques such as EWMA, Ledoit-Wolf shrinkage, and Black-Litterman blending are used to ensure stability and reliability of model inputs.


In [2]:
# try:
# The prepare_model_inputs function now internally handles the creation
# of DCF-based views if enabled in the config.
data_fetcher_instance = data_fetcher.DataFetcher(yf)
model_inputs = ip.prepare_model_inputs(config, data_fetcher_instance)
mean_returns = model_inputs.mean_returns
cov_matrix = model_inputs.cov_matrix
log_returns = model_inputs.log_returns
close_df = model_inputs.close_df
final_tickers = model_inputs.final_tickers
# except Exception as e:
#    print(f"Data pipeline failed: {e}. Cannot proceed with analysis.")
#    mean_returns, cov_matrix, log_returns, close_df, final_tickers = (
#        pd.Series(dtype=float),
#        pd.DataFrame(),
#        pd.DataFrame(),
#        pd.DataFrame(),
#        [],
#    )

2025-07-12 16:16:27 - portfolio_analyzer.data.input_preparator - INFO - --- Starting Data Pipeline for 21 tickers from 2020-07-13 to 2025-07-12 ---
2025-07-12 16:16:27 - portfolio_analyzer.data.input_preparator - INFO - DCF Views Enabled: True
2025-07-12 16:16:27 - portfolio_analyzer.data.data_fetcher - INFO - Fetching historical price data for 21 tickers...
2025-07-12 16:19:03 - yfinance - ERROR - 
13 Failed downloads:
2025-07-12 16:19:03 - yfinance - ERROR - ['SALME.OL']: Timeout('Failed to perform, curl: (28) Operation timed out after 10001 milliseconds with 6501 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
2025-07-12 16:19:03 - yfinance - ERROR - ['SAAB-B.ST']: Timeout('Failed to perform, curl: (28) Operation timed out after 10009 milliseconds with 20795 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
2025-07-12 16:19:03 - yfinance - ERROR - ['SAMPO.HE']: Timeout('Failed to perform, curl: (28

KeyError: "['KOG.OL', 'SHB-A.ST', 'SWED-A.ST'] not in index"

In [ ]:
# Visualize the distribution of daily returns for the processed assets
arithmetic_returns = log_returns.apply(lambda x: np.exp(x) - 1)
plt.figure(figsize=(14, 7))
sns.violinplot(data=arithmetic_returns, palette="Set2", inner="quartile", linewidth=1.5)
plt.title("Distribution of Daily Arithmetic Returns by Asset", fontsize=16, fontweight="bold")
plt.xlabel("Ticker", fontsize=14)
plt.ylabel("Daily Arithmetic Return", fontsize=14)
plt.xticks(rotation=45, ha="right", fontsize=11)
plt.yticks(fontsize=11)
plt.grid(True, linestyle="--", alpha=0.7, axis="y")
plt.tight_layout()
plt.show()

### 2.1. Review Model Return Estimates

After running the pipeline, compare the following annualized return estimates for each asset:

- **Historical (EWMA Shrunk):** Backward-looking returns based on historical prices.
- **Implied Equilibrium:** Forward-looking returns implied by market capitalization weights.
- **BL Posterior (Final):** The final blended return from the Black-Litterman model, combining equilibrium returns with DCF-based views. This vector is used for portfolio optimization.

In [ ]:
if "model_inputs" in locals() and model_inputs and not model_inputs.mean_returns.empty:
    returns_df = pd.DataFrame(index=final_tickers)

    # Convert annualized log returns to arithmetic
    def log_to_arith(r):
        return np.exp(r) - 1 if r is not None else None

    returns_df["Historical"] = model_inputs.hist_mean_returns.apply(log_to_arith)

    if model_inputs.implied_equilibrium_returns is not None:
        returns_df["Implied Equilibrium"] = model_inputs.implied_equilibrium_returns.apply(
            log_to_arith
        )

    returns_df["BL Posterior (Final)"] = model_inputs.mean_returns.apply(log_to_arith)

    # Display the DataFrame, formatted as percentages
    display(
        returns_df.style.format("{:.2%}")
        .background_gradient(cmap="viridis", axis=0)
        .set_caption("Comparison of Annualized Arithmetic Return Estimates")
        .set_properties(**{"text-align": "right"})
        .highlight_max(axis=0, color="lightgreen")
        .highlight_min(axis=0, color="lightcoral")
    )
else:
    print("Model inputs not available. Cannot display return estimates.")


### Visualizing Asset Correlations

Before optimization, it is essential to understand the relationships between selected assets. The correlation matrix and network graph below help assess diversification potential within the portfolio.

In [ ]:
if not cov_matrix.empty:
    correlation_matrix_df = calculate_correlation_matrix(cov_matrix)

    plot_correlation_heatmap(correlation_matrix_df)
    plot_correlation_network(correlation_matrix_df, threshold=0.3)
else:
    print("Covariance matrix is empty. Skipping correlation plots.")

## 3. Interactive Portfolio Optimization

The next step is to determine the optimal portfolio weights by maximizing the Sharpe Ratio, subject to constraints such as maximum allocation per asset. The L2 regularization parameter (`lambda_reg`) can be adjusted interactively to control overfitting and promote diversification.

In [ ]:
optimizer = PortfolioOptimizer(
    mean_returns=model_inputs.mean_returns,
    cov_matrix=model_inputs.cov_matrix,
    config=config,
)

stats_calculator = SimulationStatisticsCalculator(initial_value=config.monte_carlo.initial_value)

mc_simulator = MonteCarloSimulator(config=config, stats_calculator=stats_calculator)

session = PortfolioAnalysisSession(
    config,
    model_inputs,
    optimizer=optimizer,
    mc_simulator=mc_simulator,
)

print("--- Interactive Portfolio Optimization ---")
interact(
    session.run_interactive_optimization,
    lambda_reg=FloatSlider(
        min=0.0,
        max=2.0,
        step=0.05,
        value=config.optimization.lambda_reg,
        description="L2 Lambda:",
        continuous_update=False,
        layout={"width": "500px"},
    ),
)

## 4. Monte Carlo Simulation

To assess the potential future performance of the optimized portfolio, we run a Monte Carlo simulation. This projects thousands of possible future portfolio paths, quantifying the range of outcomes and associated risks. Simulation parameters, including the use of a Student’s t-distribution for fat tails, can be adjusted interactively.

In [ ]:
print("\n--- Monte Carlo Simulation ---")
interact(
    session.run_interactive_monte_carlo,
    num_sim_interactive=IntSlider(
        min=1000,
        max=100000,
        step=1000,
        value=config.monte_carlo.num_simulations,
        description="Num Simulations:",
        continuous_update=False,
        layout={"width": "500px"},
    ),
    time_horizon_interactive=FloatSlider(
        min=0.5,
        max=10.0,
        step=0.5,
        value=config.monte_carlo.time_horizon_years,
        description="Time Horizon (Yrs):",
        continuous_update=False,
        layout={"width": "500px"},
    ),
    df_t_interactive=IntSlider(
        min=0,
        max=30,
        step=1,
        value=config.monte_carlo.df_t_distribution,
        description="Student-t df (0-2=Normal):",
        continuous_update=False,
        layout={"width": "500px"},
    ),
)

## 5. Efficient Frontier Analysis

The efficient frontier illustrates the set of optimal portfolios offering the highest expected return for a given level of risk. This visualization highlights:

- **Maximum Sharpe Ratio Portfolio:** The tangency portfolio with the best risk-adjusted return.
- **Minimum Volatility Portfolio:** The portfolio with the lowest achievable risk.

Both are key reference points for portfolio construction.

In [ ]:
# --- Calculate and Plot the Efficient Frontier ---

print("\n--- Efficient Frontier Analysis ---")

if "session" in locals() and session.optimizer:
    try:
        frontier_df, max_sharpe_res, min_vol_res = session.optimizer.calculate_efficient_frontier()

        print("\nMaximum Sharpe Ratio Portfolio:")
        display(display_optimization_summary_html(max_sharpe_res))

        print("\nMinimum Volatility Portfolio:")
        display(display_optimization_summary_html(min_vol_res))

        # Plot the efficient frontier, highlighting the interactively found portfolio
        plot_efficient_frontier(
            frontier_df,
            max_sharpe_res,
            min_vol_res,
            current_opt_result=session.latest_result,
        )
    except Exception as e:
        print(f"Could not generate efficient frontier: {e}")
else:
    print("Analysis session not initialized. Please run the interactive optimization cell first.")

## 6. Strategy Backtesting

Finally, we conduct a historical backtest to evaluate the strategy’s real-world performance. The backtester rebalances the portfolio at a fixed frequency (e.g., quarterly), using a rolling lookback window to update optimization inputs at each step. Results are benchmarked against a relevant market index.

In [ ]:
# Define a benchmark index to compare against
# ^OSEBX is the Oslo Børs Benchmark Index


benchmark_ticker = "OSEBX.OL"

# Initialize and run the backtester
backtester = Backtester(
    config, optimizer_cls=PortfolioOptimizer, data_fetcher=data_fetcher_instance
)
backtest_results, performance_metrics = backtester.run(benchmark_ticker=benchmark_ticker)

clear_output(wait=True)

if not backtest_results.empty:
    # Display performance metrics using the new reporting function
    display(display_backtest_summary_html(performance_metrics))

    # Plot the results using the new dedicated function
    plot_backtest_results(backtest_results, benchmark_ticker)
    plt.tight_layout()
    plt.show()
else:
    print("Backtest did not produce any results.")